In [1]:
# 18.500.000 Tweets importieren

import pandas as pd
Tweetdf = pd.read_csv('emoji.txt', error_bad_lines=False, lineterminator='\n', delimiter='\n')
Tweetdf.columns = ['Tweets']

print(Tweetdf.shape)
Tweetdf.head()

c:\users\user\anaconda3\envs\capgemini\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


FileNotFoundError: [Errno 2] No such file or directory: 'emoji.txt'

In [ ]:
# Emojis aus Tweets filtern

from collections import Counter
import emoji

def extract_emojis(s):
  return ''.join(set(c for c in s if c in emoji.UNICODE_EMOJI['en']))

def count_emojis(s):
  return len([i for i in s])

Tweetdf['Emojis'] = Tweetdf['Tweets'].apply(extract_emojis)
Tweetdf = Tweetdf.drop(['Tweets'], axis=1)

myEmojis = Counter([emo for arr in Tweetdf['Emojis'].tolist() for emo in arr])

CUT = len(Tweetdf) / 10000 

def cut_rare_emojis(s):
    myList = []
    for i in s:
        if myEmojis[i] > CUT:
            myList.append(i)
    return "".join(myList)
        
Tweetdf['Emojis'] = Tweetdf['Emojis'].apply(cut_rare_emojis)

Tweetdf['nEmojis'] = Tweetdf['Emojis'].apply(count_emojis)
Tweetdf = Tweetdf[Tweetdf['nEmojis'] >= 2].reset_index(drop=True)

print(len(Tweetdf))
Tweetdf.head()

In [ ]:
# Neuen Datensatz aus Emojis erstellen

emojis = {}

for indx, row in Tweetdf.iterrows():
    for i in row['Emojis']:
        if i not in emojis:
            emojis[i] = [{j.strip():1 for j in row['Emojis']}, 1]

        else:
            emojis[i][1] += 1
            for j in row['Emojis']:
                eStrip = j.strip()
                if eStrip in emojis[i][0]:
                    emojis[i][0][eStrip] += 1

                else:
                    emojis[i][0][eStrip] = 1

for key, val in emojis.items():
    val[0].pop(key)

EmojiData = {
    'Emojis': [i for i in emojis.keys()],
    'With': [i for i, j in emojis.values()],
    'n': [j for i, j in emojis.values()]
}

Emojidf = pd.DataFrame(EmojiData)

Emojidf.head()

In [7]:
Emojidf = pd.read_csv('emoji.csv')

In [10]:
# clustering Class

from copy import deepcopy

class Cluster:
    def __init__(self, data, startEmojis):
        self.data = data
        self.startEmojis = startEmojis
        self.totalMeanDis = 1
        self.minDis = [[], 1]
        self.myClusters = self.startCluster()
    
    def startCluster(self):
        myClusters = [{i:0} for i in self.startEmojis]
        re, myClusters = self.algoCluster(myClusters, data=self.data, badLater=True)
        print('badLater!')
        totalMeanDis, myClusters = self.algoCluster(myClusters, data=re)
        self.totalMeanDis = totalMeanDis
        return myClusters
    
    def rearrangeCluster(self, n, badLater=False):
        myClusters = deepcopy(self.myClusters)
        re = []
        for cluster in myClusters:
            lastEmojis = list(cluster.keys())[n:]
            for i in lastEmojis:
                cluster.pop(i)
                lastRow = Emojidf.loc[Emojidf.index[Emojidf['Emojis'] == i].values[0]]
                re.append(lastRow)
                for key in cluster.keys():
                    cluster[key] -= self.getDistance(lastRow, key)
        
        random.shuffle(re)
        if badLater:
            re, myClusters = self.algoCluster(myClusters, data=re, badLater=True)
            print('badLater!')
        totalMeanDis, myClusters = self.algoCluster(myClusters, data=re)
        return myClusters, totalMeanDis
    
    def getDistance(self, e1, e2):
#         B = len(Tweetdf)
        B = 6558070
        N1 = e1['n']
        N2 = Emojidf.loc[Emojidf.index[Emojidf['Emojis'] == e2].values[0], 'n']
        T = eval(e1['With']).get(e2, 0)
        Z = N1 * N2 / B
        M = min(N1, N2)
        D = (T - M) / (Z - M)
        return D ** 2
    
    def algoCluster(self, myClusters, data, badLater=False):
        reList = []
        for row in data:
            emoji = row['Emojis']
            if emoji not in [e for cl in myClusters for e in cl.keys()]:
                eCluster = {idx: [] for idx in range(len(myClusters))}
                for idx, i in enumerate(myClusters):
                    for j in i.keys():
                        dis = self.getDistance(row, j)
                        if dis < self.minDis[1]:
                            self.minDis[1] = dis
                            self.minDis[0] = [row, j]
                        eCluster[idx].append(dis)

                meanDis = [sum(eCluster[i]) / len(myClusters[i]) for i in range(len(myClusters))]
                if badLater and min(meanDis) >= .8:
                    reList.append(row)

                else:
                    eIdx = meanDis.index(min(meanDis))
                    for idx, i in enumerate(myClusters[meanDis.index(min(meanDis))].keys()):
                        myClusters[eIdx][i] += eCluster[eIdx][idx]
                    myClusters[eIdx][emoji] = sum(eCluster[eIdx])
                    print(f"{emoji} -> {meanDis.index(min(meanDis)) + 1}")

        totalmeandis = 0
        for idx, i in enumerate(myClusters):
            j = dict(sorted(i.items(), key=lambda item: item[1]))
            totalmeandis += sum(i.values()) / max(len(i.values()) - 1, 1) / max(len(i.values()), 1)
            myClusters[idx] = j
        totalmeandis /= len(myClusters)
            
        if badLater:
            return reList, myClusters

        return totalmeandis, myClusters
    
    def summarizeCluster(self):
        for idx, i in enumerate(self.myClusters):
            print(f'\n{idx+1}')
            print(''.join(i.keys()))
        print(f'\n{self.totalMeanDis}')
            

In [11]:
# shuffle data and create cluster entity

import random

shuffleData = list(Emojidf.iterrows())
random.shuffle(shuffleData)
shuffleData = [row for idx, row in shuffleData]

cluster = Cluster(shuffleData, ['🤔', '♀', '🌸', '🐾', '🍰', '🎧', '🏆', '👉', '🚨', '🎥'])
print('finished!')

👮 -> 2
🏁 -> 8
🧖 -> 2
🔥 -> 8
🎷 -> 6
🏾 -> 2
🐀 -> 9
🚚 -> 3
✨ -> 3
🚧 -> 9
🥈 -> 7
🙋 -> 2
🏅 -> 7
⬆ -> 9
♨ -> 8
😸 -> 4
🎼 -> 6
🌼 -> 3
🏨 -> 8
🥉 -> 7
🤦 -> 2
🎹 -> 6
🍄 -> 3
🎉 -> 5
📼 -> 10
🤸 -> 2
🎈 -> 4
🎬 -> 10
🐈 -> 4
🌻 -> 3
🍾 -> 5


KeyboardInterrupt: 

In [ ]:
# summarize created clusters and rearrange poor connected emojis

print(cluster.totalMeanDis)
newCluster, new = cluster.rearrangeCluster(20, badLater=True)
cluster.myCluster = newCluster
newCluster, new = cluster.rearrangeCluster(-15)
while new < cluster.totalMeanDis:
    cluster.myCluster, cluster.totalMeanDis = newCluster, new
    newCluster, new = cluster.rearrangeCluster(-15)
print(cluster.summarizeCluster())
